<a href="https://colab.research.google.com/github/saaranshM/tfpractice/blob/main/Imdb_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf


In [5]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [7]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [21]:
train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

for s,l in train_data:
  train_sentences.append(s.numpy().decode('utf-8'))
  train_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(s.numpy().decode('utf-8'))
  test_labels.append(l.numpy())

train_labels_final = np.array(train_labels)
test_labels_final = np.array(test_labels)



In [22]:
vocab_size = 10000
embedding_dims = 16
max_length = 120
trunc_type = 'post'
oov_tok='<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(test_sequences, maxlen=max_length)

In [23]:
reverse_word_index = dict([(value, key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, "?") for i in text])

print(decode_review(padded[3]))
print(train_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [24]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dims, input_length=max_length),
                             tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [26]:
num_epochs = 10
model.fit(padded, train_labels_final, epochs=num_epochs, validation_data=(testing_padded, test_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5013 - accuracy: 0.7341 - val_loss: 0.3360 - val_accuracy: 0.8534
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2434 - accuracy: 0.9060 - val_loss: 0.3594 - val_accuracy: 0.8456
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0968 - accuracy: 0.9742 - val_loss: 0.4442 - val_accuracy: 0.8325
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0249 - accuracy: 0.9966 - val_loss: 0.5303 - val_accuracy: 0.8290
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0063 - accuracy: 0.9995 - val_loss: 0.5967 - val_accuracy: 0.8280
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6460 - val_accuracy: 0.8296
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 8.8971e-04 - accuracy: 1.0000 - val_loss: 0.6891 - val_accuracy: 0.8302
Ep

In [27]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [30]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [31]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
